In [99]:
# Importing libraries
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time
from datetime import date
from datetime import datetime
import hashlib
import requests
from bs4 import BeautifulSoup

Stocks = ["00001","00002","00003","00004","00005"] #Stocks that you want to keep monitor
Stock_Changed= [] #Stocks that have already sent the email notification
content = MIMEMultipart() 
content["subject"] = "Financial Result Announcement"  #Email Subject
content["from"] = "Please Fill in the Sender Email Address"  #Sender
content["to"] = "Please Fill in the Receiver Email Address" #Receiver
while True:
    for i in range(len(Stocks)):
        if Stocks[i] not in Stock_Changed:
            #Get the link 
            response = requests.get("http://www.aastocks.com/en/stocks/analysis/stock-aafn/" + Stocks[i] + "/0/result-announcement/1/")
            soup = BeautifulSoup(response.text, 'html.parser')
            #Find the Date and the link of the news
            Stock_Link = soup.find(id="cp_ucAAFNSearch_repNews_lnkNews_0").get("href")
            Date = soup.find(class_="newstime4")
            Website_Date = datetime.strptime(Date.prettify().split("\n")[2][2:], '%Y/%m/%d %H:%M')
            print("Checking Stock " + Stocks[i] +"\n" )
            try:
                if datetime.today().date() != Website_Date:
                    print("Stock " + Stocks[i] + " date : " + str(Website_Date.date()))
                    print("Toda's date : " + str(datetime.today().date()))
                    print("Nothing changed in Stock " + Stocks[i] +"\n")
                    time.sleep(10)
                    continue
                else:
                    print("Something changed in Stock " + Stocks[i] +"\n")
                    content.attach(MIMEText("Hi,\n\n" + "There is one new financial result announcement in stock number " + Stocks[i] + ". Please see the link below." + "\n" + "http://www.aastocks.com"+ Stock_Link + "\n\n" + "Regards" + "\n" + "Alpha Quant"))  #郵件內容
                    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  
                        try:
                            smtp.ehlo()  
                            smtp.starttls()  
                            smtp.login("Please Fill in the Sender Email Address", "Please Fill in your Email Password")  # Login the account
                            smtp.send_message(content)  # send email
                            print("Complete!" +"\n")
                            Stock_Changed.append(Stocks[i])
                        except Exception as e:
                            print("error" +"\n" )
                    time.sleep(10)
                    continue
            
            # To handle exceptions
            except Exception as e:
                print("error")



Checking Stock 00001

Stock 00001 date : 2021-08-05
Toda's date : 2021-11-02
Nothing changed in Stock 00001

Checking Stock 00002

Stock 00002 date : 2021-10-04
Toda's date : 2021-11-02
Nothing changed in Stock 00002



KeyboardInterrupt: 